In [ ]:
import torch
import data_collection
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import torch.nn as nn
import math
# data_collection/tiktok-trending.csv
df = pd.read_csv('../data_collection/tiktok-trending.csv')


In [ ]:

# List of features for when we do more than just the waveform
# features = ['TikTok ID','User Verified','Music ID','Music Title','Music Author','Music URL','Video Duration']
# x = df[features]
x = df['Waveform']
y = df['Play Count']

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.80, random_state=100
                                                    )
X_train = torch.tensor(x_train.values).type(torch.FloatTensor)
X_test = torch.tensor(x_test.values).type(torch.FloatTensor)

#May need to do some log transformations to make sure the scaling of outputs is workable
Y_train = torch.tensor(y_train.values).type(torch.FloatTensor)
Y_test = torch.tensor(y_test.values).type(torch.FloatTensor)


In [ ]:
class tokNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.L1 = nn.Sequential(
            nn.Conv2d(2,x_train[0].shape[1],kernel_size=(5,5),stride=(2,2),padding =(2,2)),
            nn.ReLU(),
            nn.BatchNorm2d(x_train[0].shape[1]),
            nn.init.kaiming_normal_(self.L1[0].weight, a=0.1),
            self.L1[0].bias.data.zero_()
        )
        self.L2 = nn.Sequential(
            nn.Conv2d(2,x_train[0].shape[1],kernel_size=(5,5),stride=(2,2),padding =(2,2)),
            nn.ReLU(),
            nn.BatchNorm2d(x_train[0].shape[1]),
            nn.init.kaiming_normal_(self.L2[0].weight, a=0.1),
            self.L2[0].bias.data.zero_()
        )
        self.L3 = nn.Sequential(
            nn.Conv2d(2,x_train[0].shape[1],kernel_size=(5,5),stride=(2,2),padding =(2,2)),
            nn.ReLU(),
            nn.BatchNorm2d(x_train[0].shape[1]),
            nn.init.kaiming_normal_(self.L3[0].weight, a=0.1),
            self.L3[0].bias.data.zero_()
        )
        self.L4 = nn.Sequential(
            nn.Conv2d(2,x_train[0].shape[1],kernel_size=(5,5),stride=(2,2),padding =(2,2)),
            nn.ReLU(),
            nn.BatchNorm2d(x_train[0].shape[1]),
            nn.init.kaiming_normal_(self.L4[0].weight, a=0.1),
            self.L4[0].bias.data.zero_()
        )
    
    def forward(self, x):
        
        one = self.L1(x)
        two = self.L2(one)
        three = self.L3(two)
        four = self.L4(three)
        
        return nn.linear(four)

seed = 12
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
epochs = 1000
learning_rate = 0.001
model = tokNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),learning_rate)
losses = []
testLosses = []
# threshold = 0.5

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    Y_pred = torch.squeeze(model(X_train))    
    loss = criterion(Y_pred,Y_train)
    loss.backward()
    optimizer.step()
    
with torch.no_grad():
    testRight = 0
    testTot = 0
    testOut = torch.squeeze(model(X_test))
    testLoss = criterion(testOut,Y_test)
    testPred =testOut
    testAcc =testOut.round().detach().numpy()
    testTot = Y_test.size(0)
    testRight = np.sum(testAcc == Y_test.detach().numpy())
    accuracy = 100 * testRight/testTot
    testLosses.append(testLoss.item())
    print(accuracy)